In [1]:
import yfinance as yf
import csv
import pandas as pd
import datetime

In [207]:
# GET LIST OF COMPANIES
# Get the symbol/ticker of the companies in the NASDAQ market from CSV previously downloaded from
# https://www.nasdaq.com/market-activity/stocks/screener
names = pd.read_csv('/home/jorgeluisgg/code/jorgeluisgg/project_awesome/raw_data/nasdaq_screener.csv')
print(names.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7004 entries, 0 to 7003
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      7003 non-null   object 
 1   Name        7004 non-null   object 
 2   Last Sale   7004 non-null   object 
 3   Net Change  7004 non-null   float64
 4   % Change    7004 non-null   object 
 5   Market Cap  6600 non-null   float64
 6   Country     6709 non-null   object 
 7   IPO Year    3948 non-null   float64
 8   Volume      7004 non-null   int64  
 9   Sector      6408 non-null   object 
 10  Industry    6407 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 602.0+ KB
None


In [237]:
# Create list with all tickers to iterate with the yfinance API
tickers_list = names[names.columns[0]].tolist()
tickers = [str(ticker) for ticker in tickers_list if "^" not in str(ticker)]
print(f'The number of companies available in the nasdaq list is: {len(tickers)}.')

The number of companies available in the nasdaq list is: 6622.


In [238]:
# Create list of yfinance objects
stocks = []
# Loop to fetch object for each ticker
for ticker in tickers:
    # Append the object to the list
    stocks.append(yf.Ticker(ticker))

print(f'The number of companies that were retrieved is: {len(stocks)}.')
# stocks[0:5],stocks[-5:]

The number of companies that were retrieved is: 6622.


In [228]:
# FLATTEN THE INFORMATION TO CREATE A DATAFRAME
# Create function to flatten information
def flatten_data(df,ticker):
    # Unstack the dataframe and reset the index
    df_ = df.unstack().reset_index()

    # Create the new column with row values containing the year and the feature names combined
    df_['Ticker'] = df_['level_0'].dt.year.astype(str) + ' ' + df_['level_1']

    # Drop unnecessary columns and transpose the column with names as the new head
    df_transposed = df_.drop(columns=['level_0', 'level_1']).set_index('Ticker').T

    # Add the Ticker column and set it as index
    df_transposed['Ticker'] = ticker
    df_transposed.set_index('Ticker', inplace=True)

    return df_transposed

In [288]:
# EXAMPLE OF THE FUNCTION WORKING
company = yf.Ticker("BKHA")
# flatten_data(company.cash_flow, company.ticker)
# stocks[42:44]
# stocks[42].balance_sheet
company.balance_sheet

,2023-11-30
Ordinary Shares Number,1725000.0
Share Issued,1725000.0
Net Debt,124900.0
Total Debt,250000.0
Tangible Book Value,6147.0
Invested Capital,256147.0
Working Capital,-100853.0
Net Tangible Assets,6147.0
Common Stock Equity,6147.0
Total Capitalization,6147.0


In [259]:
data = []
for stock in stocks[42:44]:
    data.append(flatten_data(stock.balance_sheet,stock.ticker))

df_cf = pd.concat(data,axis = 0, ignore_index = True)

[Ticker 2023 Ordinary Shares Number 2023 Share Issued 2023 Net Debt  \
 Ticker                                                               
 ACA                     48562279.0        48562279.0   463900000.0   
 
 Ticker 2023 Total Debt 2023 Tangible Book Value 2023 Invested Capital  \
 Ticker                                                                  
 ACA        568700000.0             1070600000.0          2900700000.0   
 
 Ticker 2023 Working Capital 2023 Net Tangible Assets 2023 Common Stock Equity  \
 Ticker                                                                          
 ACA             480800000.0             1070600000.0             2332000000.0   
 
 Ticker 2023 Total Capitalization  ... 2019 Inventory 2019 Finished Goods  \
 Ticker                            ...                                      
 ACA                 2893900000.0  ...            NaN                 NaN   
 
 Ticker 2019 Work In Process 2019 Raw Materials 2019 Receivables  \
 Ticker     

In [226]:
# df_cf.to_csv(f'/home/jorgeluisgg/code/jorgeluisgg/project_awesome/raw_data/balance_sheets.csv')
df_cf

Ticker,2023 Ordinary Shares Number,2023 Share Issued,2023 Net Debt,2023 Total Debt,2023 Tangible Book Value,2023 Invested Capital,2023 Working Capital,2023 Net Tangible Assets,2023 Common Stock Equity,2023 Total Capitalization,...,2021 Allowance For Doubtful Accounts Receivable,2021 Gross Accounts Receivable,2020 Restricted Cash,2020 Allowance For Doubtful Accounts Receivable,2020 Gross Accounts Receivable,2020 Cash Financial,2019 Restricted Cash,2019 Allowance For Doubtful Accounts Receivable,2019 Gross Accounts Receivable,2019 Cash Financial
Ticker,,,,,,,,,,,,,,,,,,,,,
A,292123241.0,292123241.0,1145000000.0,2735000000.0,1410000000.0,8580000000.0,2583000000.0,1410000000.0,5845000000.0,8580000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AA,178472464.0,178472464.0,867000000.0,1915000000.0,4068000000.0,6062000000.0,1375000000.0,4068000000.0,4251000000.0,5983000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACG,64000260.0,64585618.0,NaN,22606871.0,-142137735.0,113037868.0,-243957610.0,-142137735.0,113037868.0,113037868.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACI,8072849.0,8072849.0,2504155.0,2564439.0,16830794.0,19395233.0,-8493234.0,16830794.0,16830794.0,16830794.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACIW,8072849.0,8072849.0,2504155.0,2564439.0,16830794.0,19395233.0,-8493234.0,16830794.0,16830794.0,16830794.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACT,62500000.0,62500000.0,3094877.0,5000000.0,502674292.0,507674292.0,2135940.0,502674292.0,502674292.0,507674292.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AADI,24554205.0,24554205.0,NaN,1267000.0,105195000.0,105195000.0,98191000.0,105195000.0,105195000.0,105195000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAGR,57866830.0,57866830.0,2268740.0,11735860.0,-27010851.0,-17526396.0,-26612930.0,-27010851.0,-22583045.0,-22376758.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAGRW,57866830.0,57866830.0,2268740.0,11735860.0,-27010851.0,-17526396.0,-26612930.0,-27010851.0,-22583045.0,-22376758.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# data = []
# for stock in stocks:
#     data.append(flatten_data(stock.income_stmt,stock.ticker))
# df_cf = pd.concat(data,axis = 0)

# df_cf.to_csv(f'/home/jorgeluisgg/code/jorgeluisgg/project_awesome/raw_data/income_statements.csv')

In [ ]:
# data = []
# for stock in stocks:
#     data.append(flatten_data(stock.cash_flow,stock.ticker))
# df_cf = pd.concat(data,axis = 0)

# df_cf.to_csv(f'/home/jorgeluisgg/code/jorgeluisgg/project_awesome/raw_data/cash_flows.csv')

In [297]:
import requests
import time

In [299]:


def fetch_data_with_retries(url, max_retries=5):
    retries = 0
    wait_time = 1  # Initial wait time in seconds

    while retries < max_retries:
        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            print(f"Rate limited. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            retries += 1
            wait_time *= 2  # Exponential backoff
        else:
            print(f"Failed to retrieve data: {response.status_code}")
            return None

    print("Max retries reached. Failed to retrieve data.")
    return None

# url = 'https://query2.finance.yahoo.com/v8/finance/chart/^GSPC'
target = '^GSPC'
url = f"https://api.iex.cloud/v1/data/core/cash_flow/{target}/quarterly?last=20&token=sk_bfc07c66f7754ae79fc6eec48e6428e2"
api_data = fetch_data_with_retries(url)

if api_data:
    print(api_data)

Failed to retrieve data: 402
